In [1]:
#from nltk import download

#download()

TODO: обучить модель на корпусе текстов с интернет-сленгом

In [2]:
#from nltk import sent_tokenize
#from gensim.utils import tokenize
#from gensim.models import *

#SKIP FOR NOW

'''
with open('2ch.txt') as f:
    text = f.read().replace('\n', '')
    
sentences = [list(tokenize(sent)) for sent in sent_tokenize(text)]

min_count = 1
size = 50
window = 10

model = Word2Vec(sentences, min_count=min_count, size=size, window=window,  workers=4)
'''

"\nwith open('2ch.txt') as f:\n    text = f.read().replace('\n', '')\n    \nsentences = [list(tokenize(sent)) for sent in sent_tokenize(text)]\n\nmin_count = 1\nsize = 50\nwindow = 10\n\nmodel = Word2Vec(sentences, min_count=min_count, size=size, window=window,  workers=4)\n"

Утилсы: регулярки, загрузка файлов и т.д.

In [22]:
from re import sub, compile

def cut(data):
    r = compile(r'<.*?>|>>.*|&#(47|92);|&quot;|&gt;|(http|https):.*')
    return r.sub('', data)

def pos_cut(s):
    s = sub(r"ADJF", "ADJ", s)
    s = sub(r"ADVB", "ADV", s)
    s = sub(r"INFN", "VERB", s)
    return s

def topic_cut(word):
    return sub('[^a-zA-Zа-яА-Я]', '', word)

def check_for_rus_lan(word):
    regex = compile('[А-Яа-я]+$')
    if regex.match(word):
        return True
    return False

Загрузка каталога тредов выбранной доски

In [5]:
from requests import get

DVACH = 'https://2ch.hk/'
BOARD = 'pr'

dvach_page = get(DVACH + BOARD + '/catalog.json').json()
threads = [i['num'] for i in dvach_page['threads']]

Загрузка n-ного треда из списка

In [6]:
thread = get(DVACH +  BOARD + '/res/' + threads[5] + '.json').json()
thread['title']

'C Programming Language #20'

Подготовка корпуса

In [7]:
from gensim import corpora
from nltk.corpus import stopwords

documents = [cut(i['comment']) for i in thread['threads'][0]['posts'] if cut(i['comment'])]

texts = [[word for word in document.lower().split() if word not in stopwords.words('russian') and word.isalpha() and len(word) > 3]
         for document in documents]

# remove words that appear only once
all_tokens = sum(texts, [])
tokens_once = set(word for word in set(all_tokens) if all_tokens.count(word) == 1)
texts = [[word for word in text if word not in tokens_once]
         for text in texts]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

Создание LDA модели

In [8]:
from gensim import models, corpora

lda = models.LdaModel(corpus, id2word=dictionary, num_topics=5)
lda.print_topics(5)

[(0,
  '0.256*"сообщения" + 0.256*"первого" + 0.251*"почему" + 0.140*"строки" + 0.024*"типа" + 0.024*"посчитать" + 0.024*"return" + 0.024*"result"'),
 (1,
  '0.130*"почему" + 0.129*"типа" + 0.126*"посчитать" + 0.123*"строки" + 0.123*"return" + 0.123*"первого" + 0.123*"сообщения" + 0.123*"result"'),
 (2,
  '0.602*"посчитать" + 0.059*"почему" + 0.058*"типа" + 0.056*"строки" + 0.056*"сообщения" + 0.056*"return" + 0.056*"первого" + 0.056*"result"'),
 (3,
  '0.256*"result" + 0.255*"return" + 0.255*"типа" + 0.139*"строки" + 0.024*"почему" + 0.024*"посчитать" + 0.024*"первого" + 0.024*"сообщения"'),
 (4,
  '0.452*"типа" + 0.081*"почему" + 0.079*"посчитать" + 0.078*"строки" + 0.078*"return" + 0.078*"первого" + 0.077*"сообщения" + 0.077*"result"')]

Создание LSI модели

In [9]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=1) # initialize an LSI transformation
corpus_lsi = lsi[corpus]
lda.print_topics(5)

[(0,
  '0.256*"сообщения" + 0.256*"первого" + 0.251*"почему" + 0.140*"строки" + 0.024*"типа" + 0.024*"посчитать" + 0.024*"return" + 0.024*"result"'),
 (1,
  '0.130*"почему" + 0.129*"типа" + 0.126*"посчитать" + 0.123*"строки" + 0.123*"return" + 0.123*"первого" + 0.123*"сообщения" + 0.123*"result"'),
 (2,
  '0.602*"посчитать" + 0.059*"почему" + 0.058*"типа" + 0.056*"строки" + 0.056*"сообщения" + 0.056*"return" + 0.056*"первого" + 0.056*"result"'),
 (3,
  '0.256*"result" + 0.255*"return" + 0.255*"типа" + 0.139*"строки" + 0.024*"почему" + 0.024*"посчитать" + 0.024*"первого" + 0.024*"сообщения"'),
 (4,
  '0.452*"типа" + 0.081*"почему" + 0.079*"посчитать" + 0.078*"строки" + 0.078*"return" + 0.078*"первого" + 0.077*"сообщения" + 0.077*"result"')]

Получение списка характеризующих тему слов

In [10]:
topics = lda.print_topic(1)
keywords= [topic_cut(word) for word in topics.split('+')]
keywords

['почему',
 'типа',
 'посчитать',
 'строки',
 'return',
 'первого',
 'сообщения',
 'result']

Загрузка обученной модели RusVectores

In [11]:
model = models.KeyedVectors.load_word2vec_format('corpora.bin', binary=True)

Предобработчик токенов для использования модели RusVectores

In [25]:
import pymorphy2 

morph = pymorphy2.MorphAnalyzer()

def process(s):
    if not check_for_rus_lan(s):
        return None
    s += '_' + morph.parse(s)[0].tag.POS
    s = pos_cut(s)
    return s

Расстояние от слова до ключевых слов тематического кластера

In [26]:
similarities = [[model.similarity(process('программирование'), process(keyword)), keyword] for keyword in keywords if process(keyword) in model.vocab]
similarities

[[0.083447638458197956, 'посчитать']]

#TODO суммарное расстояние всего сообщения